# Risk Flow Matrix Modeling

In [10]:
import pandas as pd
import numpy as np
import math
from collections import defaultdict

## Data Preprocessing

### Read the preprocessed dataset:
 - "origin_dest_trips_census_tract_level.csv"
 - "tracts_4.csv"

In [53]:
# this dataset is for the nested hashmap
OD_p1_df = pd.read_csv('Streetlight_Data/clean_data/streetlight_OD_p1.csv')[['RESIDENCE','WORKPLACE','JOBS']]
OD_p2_df = pd.read_csv('Streetlight_Data/clean_data/streetlight_OD_p2.csv')[['RESIDENCE','WORKPLACE','JOBS']]
OD_p3_df = pd.read_csv('Streetlight_Data/clean_data/streetlight_OD_p3.csv')[['RESIDENCE','WORKPLACE','JOBS']]
OD_p4_df = pd.read_csv('Streetlight_Data/clean_data/streetlight_OD_p4.csv')[['RESIDENCE','WORKPLACE','JOBS']]

In [54]:
ct_info_cases_data = pd.read_csv("Statistical_Analysis/data/ct_info_cases_data.csv")
ct_cases_df = ct_info_cases_data[['GEOID','countyFIPS','estimate_cases1','estimate_cases2','estimate_cases3','estimate_cases4']]
ct_cases_df

,GEOID,countyFIPS,estimate_cases1,estimate_cases2,estimate_cases3,estimate_cases4
0,17091011700,17091,0.030119,27.107335,38.311700,116.471181
1,17091011800,17091,0.023156,20.840201,29.454151,89.543398
2,17119400951,17119,0.018442,10.548614,17.187603,142.941103
3,17119400952,17119,0.012385,7.084098,11.542621,95.994478
4,17135957500,17135,0.000000,4.240201,4.783816,74.692765
...,...,...,...,...,...,...
3118,17037000100,17037,0.000000,25.466793,35.104603,177.373983
3119,17037001500,17037,0.000000,14.444589,19.911088,100.605297
3120,17037000400,17037,0.000000,32.713750,45.094142,227.848402
3121,17037000300,17037,0.000000,10.168505,14.016736,70.822746


### Create a dictionary: to record each origin->destination: number of trips

use the dataset: OD_census_tract_level 

1. Generate a unique set of origins in commute trips

2. Generate a doubly nested hashmap (python dictionary: key-value pair): \
 first mapping: **origin -> destination** \
 second mapping: **destination -> number of trips**

In [77]:
#function to fo defaultdict(defaultdict(int))
# when {} is empty, enable
# df['a']['b']+=1
def defaultdict_float():
    return defaultdict(float)

In [6]:
# anuallly aggregated data 
# origin -> destination (county level) frequency dictionary
# rename the origin-> destination df
OD = OD_p1_df

nu_OD_DblMap = defaultdict(defaultdict_float)
num_ct = OD_p1_df.shape[0]

for i in range(num_ct):
    fip_o = OD.iloc[i,0]
    fip_d = OD.iloc[i,1]
    num_trips = OD.iloc[i,2]   
    num_cases = float(ct_cases_df[ct_cases_df['GEOID'] == fip_o]['estimate_cases1'])
    if num_cases == 0:
        continue
    else:
        nu_OD_DblMap[fip_o][fip_d] = num_trips * num_cases

KeyboardInterrupt: 

In [ ]:
de_OD_Map = defaultdict(float)
for fip_d in nu_OD_DblMap:
    sum = 0
    for fip_o in nu_OD_DblMap:
        sum += nu_OD_DblMap[fip_o][fip_d]
    de_OD_Map[fip_d] = sum  

In [23]:
Risk_DblMap = defaultdict(defaultdict_float)
for fip_o in nu_OD_DblMap:
    for fip_d in nu_OD_DblMap[fip_o]:
        nu = nu_OD_DblMap[fip_o][fip_d]
        de = de_OD_Map[fip_d]
        if de == 0 or nu == 0:
            continue
        else:               
            Risk_DblMap[fip_o][fip_d] = nu/de

In [59]:
def getTotalRiskflow(OD, estimate_cases):
    total_riskflow = 0
    num_ct = OD.shape[0]
    for i in range(num_ct):
        fip_o = OD.iloc[i,0]
        fip_d = OD.iloc[i,1]
        num_trips = OD.iloc[i,2]   
        num_cases = float(ct_cases_df[ct_cases_df['GEOID'] == fip_o][estimate_cases])
        total_riskflow += num_trips*num_cases/10**6
    return total_riskflow

In [60]:
total_riskflow1 = getTotalRiskflow(OD = OD_p1_df, 
                 estimate_cases = 'estimate_cases1')
total_riskflow2= getTotalRiskflow(OD = OD_p2_df, 
                 estimate_cases = 'estimate_cases2')
total_riskflow3 = getTotalRiskflow(OD = OD_p3_df, 
                 estimate_cases = 'estimate_cases3')
total_riskflow4 = getTotalRiskflow(OD = OD_p4_df, 
                 estimate_cases = 'estimate_cases4')

AttributeError: 'numpy.float64' object has no attribute 'keys'

In [161]:
# new
def getDPTotalRisk(OD, estimate_cases, apple_mobility):
    nu_OD_DblMap = defaultdict(defaultdict_float)
    num_ct = OD.shape[0]
    total_riskflow = 0

    for i in range(num_ct):
        fip_o = OD.iloc[i,0]
        fip_d = OD.iloc[i,1]
        num_trips = OD.iloc[i,2]   
        num_cases = float(ct_cases_df[ct_cases_df['GEOID'] == fip_o][estimate_cases])
        total_riskflow += num_trips * num_cases 
        if num_cases == 0:
            continue
        else:
            nu_OD_DblMap[fip_o][fip_d] = num_trips * num_cases
        
    for fip_o in nu_OD_DblMap:
        for fip_d in nu_OD_DblMap[fip_o]:
            if nu_OD_DblMap[fip_o][fip_d] == 0:
                continue
            else:
                nu_OD_DblMap[fip_o][fip_d] = nu_OD_DblMap[fip_o][fip_d]/total_riskflow * apple_mobility
    print(total_riskflow)       
    return nu_OD_DblMap

In [231]:
len(OD_p4_df.iloc[:,0].unique())

3116

In [122]:
dptotal_Risk_DblMap1 =  getDPTotalRisk(OD = OD_p1_df, 
                                         estimate_cases = 'estimate_cases1',
                                         apple_mobility = 109.278621/100)
dptotal_Risk_DblMap2 =  getDPTotalRisk(OD = OD_p2_df,
                                         estimate_cases = 'estimate_cases2',
                                         apple_mobility = 73.2133333/100)
dptotal_Risk_DblMap3 =  getDPTotalRisk(OD = OD_p3_df,
                                         estimate_cases = 'estimate_cases3',
                                         apple_mobility = 130.970323/100)
dptotal_Risk_DblMap4 =  getDPTotalRisk(OD = OD_p4_df,
                                         estimate_cases = 'estimate_cases4',
                                         apple_mobility = 139.87/100)

In [166]:
sum = 0
count = 0
for i in dptotal_Risk_DblMap3 :
    for j in dptotal_Risk_DblMap3[i]:
        if dptotal_Risk_DblMap3[i][j] == 0:
            count += 1
        sum += dptotal_Risk_DblMap3[i][j]
count

19

In [123]:
list_dptotal_Risk_DblMap = [dptotal_Risk_DblMap1, dptotal_Risk_DblMap2, 
                             dptotal_Risk_DblMap3, dptotal_Risk_DblMap4]

### Transfer double risk map to csv file and save to path

In [127]:
def saveRiskflowToCSV(Risk_DblMap, path):
    riskflow_df = pd.DataFrame(columns = ('START', 'END', 'RISK_FLOW'))
    for fip_o in Risk_DblMap:
        for fip_d in Risk_DblMap[fip_o]:
            riskflow = Risk_DblMap[fip_o][fip_d]
        new_row = pd.DataFrame({'START':[fip_o], 'END':[fip_d], 'RISK_FLOW': [riskflow] })
        riskflow_df = riskflow_df.append(new_row)
    riskflow_df.to_csv(path)

In [128]:
saveRiskflowToCSV(Risk_DblMap = dptotal_Risk_DblMap1, 
                  path = 'Streetlight_Data/clean_data/dptotal/dptotal_riskflow1_df.csv')
saveRiskflowToCSV(Risk_DblMap = dptotal_Risk_DblMap2, 
                  path = 'Streetlight_Data/clean_data/dptotal/dptotal_riskflow2_df.csv')
saveRiskflowToCSV(Risk_DblMap = dptotal_Risk_DblMap3, 
                  path = 'Streetlight_Data/clean_data/dptotal/dptotal_riskflow3_df.csv')
saveRiskflowToCSV(Risk_DblMap = dptotal_Risk_DblMap4, 
                  path = 'Streetlight_Data/clean_data/dptotal/dptotal_riskflow4_df.csv')

### Calculaye exposure entropy

In [177]:
def getExposure(Risk_DblMap):
#     reverse_Risk_DblMap = defaultdict(defaultdict_float)
#     for fip_o in Risk_DblMap:
#         for fip_d in Risk_DblMap[fip_o]:
#             risk = Risk_DblMap[fip_o][fip_d]   
#             if risk == 0:
#                 continue
#             else:
#                 reverse_Risk_DblMap[fip_d][fip_o] = risk

    Exposure_Map = {}
    for fip_d in Risk_DblMap:
        v = np.array(list(Risk_DblMap[fip_d].values()))
        v[v == 0] = 10**-10
        logv = np.log(v)
        entropy = - np.sum(v * logv)
        Exposure_Map[fip_d] = entropy
    return Exposure_Map

-----

In [201]:
Exposure_Map1 = getExposure(dptotal_Risk_DblMap1)
Exposure_Map2 = getExposure(dptotal_Risk_DblMap2)
Exposure_Map3 = getExposure(dptotal_Risk_DblMap3)
Exposure_Map4 = getExposure(dptotal_Risk_DblMap4)

In [232]:
exposure_df

,FIPS,exposure1,exposure2,exposure3,exposure4
0,17001000100,0.001941,0.000319,0.000838,0.006664
0,17001000201,0.000384,0.000062,0.000164,0.001351
0,17001000202,0.000660,0.000107,0.000283,0.002302
0,17001000400,0.000880,0.000143,0.000377,0.003059
0,17001000500,0.000418,0.000067,0.000179,0.001462
...,...,...,...,...,...
0,17203030501,0.010959,0.000454,0.001102,0.007808
0,17203030502,0.001048,0.000041,0.000101,0.000741
0,17203030601,0.008321,0.000359,0.000861,0.005981
0,17203030602,0.002334,0.000094,0.000230,0.001671


In [202]:
exposure_df = pd.DataFrame(columns = ('FIPS', 'exposure1', 'exposure2', 'exposure3', 'exposure4'))

In [205]:
np.sum(exposure_df['exposure4'])

11.84035746719779

In [204]:
for i, fip_d in enumerate(Exposure_Map1): 
    FIPS = str(fip_d)
    exposure1 = Exposure_Map1[fip_d]
    exposure2 = Exposure_Map2[fip_d]
    exposure3 = Exposure_Map3[fip_d]
    exposure4 = Exposure_Map4.get(fip_d, 0) 
    new_row = pd.DataFrame({'FIPS':[FIPS], 'exposure1':[exposure1],
                        'exposure2':[exposure2], 'exposure3':[exposure3],
                        'exposure4':[exposure4]})
    exposure_df = exposure_df.append( new_row )

In [199]:
exposure_df.to_csv('Streetlight_Data/clean_data/dptotal/dptotal_exposure_df.csv')

--------


In [175]:
def saveExposureToCSV(list_Risk_DblMap, path):
    Exposure_Map1 = getExposure(list_Risk_DblMap[0])
    Exposure_Map2 = getExposure(list_Risk_DblMap[1])
    Exposure_Map3 = getExposure(list_Risk_DblMap[2])
    Exposure_Map4 = getExposure(list_Risk_DblMap[3])
    
    exposure_df = pd.DataFrame(columns = ('FIPS', 'exposure1', 'exposure2', 'exposure3', 'exposure4'))
    
    for i, fip_d in enumerate(Exposure_Map1): 
        FIPS = str(fip_d)
        exposure1 = Exposure_Map1[fip_d]
        exposure2 = Exposure_Map2[fip_d]
        exposure3 = Exposure_Map3[fip_d]
        exposure4 = Exposure_Map4.get(fip_d, 0) 
        new_row = pd.DataFrame({'FIPS':[FIPS], 'exposure1':[exposure1],
                            'exposure2':[exposure2], 'exposure3':[exposure3],
                            'exposure4':[exposure4]})
        exposure_df = exposure_df.append(new_row)
    
    exposure_df.to_csv(path)

In [176]:
saveExposureToCSV(list_dptotal_Risk_DblMap, 'Streetlight_Data/clean_data/dptotal/dptotal_exposure_df.csv')